In [2]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
# 시계열 데이터를 LSTM으로 예측
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt

In [12]:
# 데이터 준비
# yfinance 라이브러리로 애플 주식 데이터 사용
# 종가:
# - 일반적으로 주가예측에 자주 사용되는 지표
# - 하루 동안의 최종 거래 가걱ㄱ을 나타내며 전체적인 추세를 반영하는데 사용
!pip install yfinance

'pip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [6]:
import yfinance as yf
df = yf.download("AAPL", start ="2015-01-01", end="2020-01-01")

ModuleNotFoundError: No module named 'yfinance'

In [33]:
df = pd.read_csv("../Data/apple.csv",header=2)
df.columns =  ['Date','Close','1','2','3','4']

df.head()

,Date,Close,1,2,3,4
0,2015-01-02,24.288584,24.757338,23.848709,24.746230,212818400
1,2015-01-05,23.604328,24.137509,23.417716,24.057531,257142000
2,2015-01-06,23.606548,23.866473,23.244429,23.668752,263188400
3,2015-01-07,23.937574,24.037545,23.704307,23.815387,160423600
4,2015-01-08,24.857307,24.915069,24.148621,24.266367,237458000


In [34]:
df = df['Close']
df.head()

0    24.288584
1    23.604328
2    23.606548
3    23.937574
4    24.857307
Name: Close, dtype: float64

In [35]:
df.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1258 entries, 0 to 1257
Series name: Close
Non-Null Count  Dtype  
--------------  -----  
1258 non-null   float64
dtypes: float64(1)
memory usage: 10.0 KB


In [41]:
df1 = pd.read_csv("../Data/apple.csv",header=0)
df1.head()
df1.drop([0,1],axis=0)

,Price,Close,High,Low,Open,Volume
2,2015-01-02,24.288583755493164,24.75733822078495,23.848709278537115,24.746229620306494,212818400
3,2015-01-05,23.604328155517578,24.137508612840765,23.417715927668134,24.05753145949559,257142000
4,2015-01-06,23.606548309326172,23.866473189260027,23.244429090081105,23.668752380257303,263188400
5,2015-01-07,23.93757438659668,24.03754501696595,23.704307322488965,23.815386553981295,160423600
6,2015-01-08,24.85730743408203,24.915069102408545,24.148621476140704,24.26636687175016,237458000
...,...,...,...,...,...,...
1255,2019-12-24,68.7328872680664,68.88280162309742,68.40648024886319,68.83444120163395,48478800
1256,2019-12-26,70.09657287597656,70.11349976170649,68.83686282423325,68.86587612619056,93121200
1257,2019-12-27,70.0699691772461,71.07822564604481,69.66376821117122,70.38913024148955,146266000
1258,2019-12-30,70.48584747314453,70.76874192675591,68.96259007950816,69.98776610931984,144114400


In [44]:

df1 = df1.rename(columns={'Price': 'Data'})
df1.drop([0,1],axis=0)

,Data,Close,High,Low,Open,Volume
2,2015-01-02,24.288583755493164,24.75733822078495,23.848709278537115,24.746229620306494,212818400
3,2015-01-05,23.604328155517578,24.137508612840765,23.417715927668134,24.05753145949559,257142000
4,2015-01-06,23.606548309326172,23.866473189260027,23.244429090081105,23.668752380257303,263188400
5,2015-01-07,23.93757438659668,24.03754501696595,23.704307322488965,23.815386553981295,160423600
6,2015-01-08,24.85730743408203,24.915069102408545,24.148621476140704,24.26636687175016,237458000
...,...,...,...,...,...,...
1255,2019-12-24,68.7328872680664,68.88280162309742,68.40648024886319,68.83444120163395,48478800
1256,2019-12-26,70.09657287597656,70.11349976170649,68.83686282423325,68.86587612619056,93121200
1257,2019-12-27,70.0699691772461,71.07822564604481,69.66376821117122,70.38913024148955,146266000
1258,2019-12-30,70.48584747314453,70.76874192675591,68.96259007950816,69.98776610931984,144114400


In [46]:
df1 = df1.drop([0,1],axis=0)
df1

,Data,Close,High,Low,Open,Volume
2,2015-01-02,24.288583755493164,24.75733822078495,23.848709278537115,24.746229620306494,212818400
3,2015-01-05,23.604328155517578,24.137508612840765,23.417715927668134,24.05753145949559,257142000
4,2015-01-06,23.606548309326172,23.866473189260027,23.244429090081105,23.668752380257303,263188400
5,2015-01-07,23.93757438659668,24.03754501696595,23.704307322488965,23.815386553981295,160423600
6,2015-01-08,24.85730743408203,24.915069102408545,24.148621476140704,24.26636687175016,237458000
...,...,...,...,...,...,...
1255,2019-12-24,68.7328872680664,68.88280162309742,68.40648024886319,68.83444120163395,48478800
1256,2019-12-26,70.09657287597656,70.11349976170649,68.83686282423325,68.86587612619056,93121200
1257,2019-12-27,70.0699691772461,71.07822564604481,69.66376821117122,70.38913024148955,146266000
1258,2019-12-30,70.48584747314453,70.76874192675591,68.96259007950816,69.98776610931984,144114400


In [48]:
df1 = df1.drop(['High', 'Low', 'Open', 'Volume'],axis=1)
df1

,Data,Close
2,2015-01-02,24.288583755493164
3,2015-01-05,23.604328155517578
4,2015-01-06,23.606548309326172
5,2015-01-07,23.93757438659668
6,2015-01-08,24.85730743408203
...,...,...
1255,2019-12-24,68.7328872680664
1256,2019-12-26,70.09657287597656
1257,2019-12-27,70.0699691772461
1258,2019-12-30,70.48584747314453


In [47]:
df1.columns

Index(['Data', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')

In [49]:
df = df1.copy()
df = df.reset_index(drop=True)

In [50]:
df

,Data,Close
0,2015-01-02,24.288583755493164
1,2015-01-05,23.604328155517578
2,2015-01-06,23.606548309326172
3,2015-01-07,23.93757438659668
4,2015-01-08,24.85730743408203
...,...,...
1253,2019-12-24,68.7328872680664
1254,2019-12-26,70.09657287597656
1255,2019-12-27,70.0699691772461
1256,2019-12-30,70.48584747314453


In [ ]:
#데이터 전처리
# : LSTM은 데이터의 스케일에 민감하므로 MinMaxScaler를 사용해 데이터 정규화

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)

scaled_data

ValueError: could not convert string to float: '2015-01-02'

In [ ]:
train_data_len = int(np.ceil(len(scaled_data))*0.8)


# 학습데이터
train_data = scaled_data[0:int(train_data), :]
test_data = scaled_data[train_data_len - 60:, :]

NameError: name 'scaled_data' is not defined

In [ ]:
int(np.ceil(len(scaled_data)*0,8))

In [ ]:
# LSTM 모델은 과거 데이터를 사용해 다음 값을 예측하도록 설례
# 60일간의 데이터를 입력으로 사용하여 다음날의 주가를 예측하는 방식
# 학습 데이터 셋
X_train = []
y_train = []

for i in range(60, len(train_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

X_train, y_train = np.array(X_train), np.array(y_train)    

# LSTM 입력을 위한 3차원 형식 변환
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


In [ ]:
# LSTM 모델 정의 및 학습
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1],1))
model.add(LSTM(50))
model.add(Dense(25))  
model.add(Dense(1))  
model.compile(optimizer= 'adam' , loss='mse', metrics=['mse'])  

history = model.fit(
    X_train, y_train,         
    epochs=5
)

In [ ]:
# 테스트 데이터 준비 및 예측
X_test = []
y_test = scaled_data[train_data_len:, :]

for i in range(60, len(test_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])

X_test = np.array(X_test)

# LSTM 입력을 위한 3차원 형식 변환
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
#예측과 정규화 전 데이터로 돌리기
predictions = model.predict(X_test) 
predictions = scaler.inverse_transform(predictions)

In [ ]:
#평가
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

In [ ]:
# 실제 주가와 예측된 주가 준비
actual_prices = df[train_data_len:]['Close'].values
predicted_prices = predictions.flatten()

In [ ]:
# 평가 지표 계산
mse = mean_squared_error(actual_prices, predicted_prices)
rmse = np.sqrt(mse)
mae = mean_absolute_error(actual_prices, predicted_prices)

In [ ]:
# 평가 결과 출력
print(f"Mean Squard Error(MSE):{mse:.2f}")
print(f"Rood Mean Squard Error(RMSE):{rmse:.2f}")
print(f"Mean Absolute Error(MAE):{mae:.2f}")


In [ ]:
# 결과 시각화
:모델 예측 결과와 실제 주가를 비교
train = df[:train_data_len]
valid = df[train_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,8))
plt.title('LSTM Model')
Plt.xlabel('Date')
plt.ylabel('Closed Price USD')

plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','val','Predictions'])
plt.show()